In [2]:
# conda env : freecad_fem_env 

# create shape (size, material, fixed constraints)
# define and apply force
# calculate deformations 
# save mesh as ply or obj file

import os
import sys

sys.path.append(os.path.join(os.environ["CONDA_PREFIX"], "lib"))

In [4]:
import FreeCAD
import Part
import ObjectsFem
import Mesh
import Fem
import femmesh.femmesh2mesh
from femtools import ccxtools

doc = App.newDocument("doc")

box_obj = doc.addObject('Part::Box', 'Box')
box_obj.Height = 2
box_obj.Width = 5
box_obj.Length = 50

analysis_object = ObjectsFem.makeAnalysis(FreeCAD.ActiveDocument, 'Analysis')

femmesh_obj = doc.addObject('Fem::FemMeshShapeNetgenObject', 'CubeFemMesh')
femmesh_obj.Shape = box_obj 
femmesh_obj.MaxSize = 1

doc.Analysis.addObject(femmesh_obj)

doc.recompute()

doc.addObject("Fem::ConstraintFixed","FemConstraintFixed")
doc.FemConstraintFixed.Scale = 1
doc.Analysis.addObject(doc.FemConstraintFixed)
for amesh in doc.Objects:
    if "FemConstraintFixed" == amesh.Name:
        print('amesh.ViewObject.Visibility = True')
    elif "Mesh" in amesh.TypeId:
        aparttoshow = amesh.Name.replace("_Mesh","")
        for apart in doc.Objects:
            if aparttoshow == apart.Name:
                print('apart.ViewObject.Visibility = True')
        print('amesh.ViewObject.Visibility = False')

doc.recompute()

doc.FemConstraintFixed.Scale = 1
doc.FemConstraintFixed.References = [(doc.Box,"Face2")]
doc.recompute()

doc.addObject("Fem::ConstraintForce","FemConstraintForce")
doc.FemConstraintForce.Force = 1.0
doc.FemConstraintForce.Reversed = False
doc.FemConstraintForce.Scale = 1
doc.Analysis.addObject(doc.FemConstraintForce)

for amesh in doc.Objects:
    if "FemConstraintForce" == amesh.Name:
        print("amesh.ViewObject.Visibility = True")
    elif "Mesh" in amesh.TypeId:
        aparttoshow = amesh.Name.replace("_Mesh","")
        for apart in doc.Objects:
            if aparttoshow == apart.Name:
                print("apart.ViewObject.Visibility = True")
        print("amesh.ViewObject.Visibility = False")

doc.recompute()
doc.FemConstraintForce.Force = 4000
doc.FemConstraintForce.Direction = None
doc.FemConstraintForce.Reversed = True
doc.FemConstraintForce.Scale = 1
doc.FemConstraintForce.References = [(doc.Box,"Face4")]
doc.recompute()

# FemGui.getActiveAnalysis().addObject(ObjectsFem.makeSolverCalculixCcxTools(doc))
analysis_object.addObject(ObjectsFem.makeSolverCalculixCcxTools(doc))
## material
material_object = ObjectsFem.makeMaterialSolid(doc, "SolidMaterial")
mat = material_object.Material
mat['Name'] = "Steel-Generic"
mat['YoungsModulus'] = "210000 MPa"
mat['PoissonRatio'] = "0.30"
mat['Density'] = "7900 kg/m^3"
material_object.Material = mat
# FemGui.getActiveAnalysis().addObject(material_object)
analysis_object.addObject(material_object)

# run the analysis step by step

fea = ccxtools.FemToolsCcx()
fea.update_objects()
fea.setup_working_dir()
fea.setup_ccx()
message = fea.check_prerequisites()
if not message:
    fea.purge_results()
    fea.write_inp_file()
    # on error at inp file writing, the inp file path "" was returned (even if the file was written)
    # if we would write the inp file anyway, we need to again set it manually
    # fea.inp_file_name = '/tmp/FEMWB/FEMMeshGmsh.inp'
    fea.ccx_run()
    fea.load_results()
else:
    FreeCAD.Console.PrintError("Houston, we have a problem! {}\n".format(message))  # in report view
    print("Houston, we have a problem! {}\n".format(message))  # in python console

# show some results
for m in analysis_object.Group:
    if m.isDerivedFrom('Fem::FemResultObject'):
        result_object = m
        break

# femmesh_obj = doc.getObject("FEMMeshGmsh")
femmesh_obj = doc.getObject("ResultMesh").FemMesh
result = doc.getObject("CCX_Results")

out_mesh = femmesh.femmesh2mesh.femmesh_2_mesh(femmesh_obj, result)
Mesh.show(Mesh.Mesh(out_mesh))

#export one mesh as .ply or obj file
__objs__=[]
__objs__.append(doc.getObject("Mesh"))

path ="/Users/aidafarahani/MyRepo/FreeCAD_Conda/" 
filename = "mymesh.obj"

Mesh.export(__objs__,os.path.join(path,filename))


apart.ViewObject.Visibility = True
amesh.ViewObject.Visibility = False
amesh.ViewObject.Visibility = True
apart.ViewObject.Visibility = True
amesh.ViewObject.Visibility = False
amesh.ViewObject.Visibility = True
CCX_Results
Mesh by surface search method:  5.3218
